<a href="https://colab.research.google.com/github/halyusa16/dbt-retail-data/blob/main/etl_retail_dbt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.oauth2 import service_account
from google.auth.transport.requests import AuthorizedSession
import gspread
import pandas as pd
from google.cloud import bigquery
import pandas_gbq

In [ ]:
scope = ['https://www.googleapis.com/auth/spreadsheets']

# adding the credential file to allow gsheet access from google API
credentials = service_account.Credentials.from_service_account_file('/content/gsheet-viewer.json', scopes = scope)

# specify the spreadsheet that we worked on
gc = gspread.Client(auth=credentials)
gc.session = AuthorizedSession(credentials)

In [ ]:
# add and open google sheet using the google sheet ID
sheet_key = '1XSsij8C-sL8p4c6cq2fWT2fXN4W0HSkv6EImSF_UNH8'
sheet = gc.open_by_key(sheet_key)

In [ ]:
# E for Extract process started here
# access the worksheet
worksheet_name = 'online_retail'
worksheet = sheet.worksheet(worksheet_name)

# store data into sheet_data variable
sheet_data = worksheet.get_all_records()

In [ ]:
# since this is an ingestion process, we will not proceed to transform and clean the data
# we just needed to store data into pandas, then later we can push it to BigQuery
df = pd.DataFrame(sheet_data)

In [ ]:
# time to push the data to BigQuery
# set the BigQuery dataset and table name
# to avoid data type issues, we will use table schema to state the data types

df = df.astype(str)  # convert all columns to string to match schema

# enter Google Cloud project ID
project_id = 'sound-chalice-349712'

# enter BigQuery dataset and create a table name
dataset_name = 'halyusa_data'
table_name = 'online_retail_data'

# construct the full table ID
table_id = f'{project_id}.{dataset_name}.{table_name}'

# push data to BQ using to_gbq
pandas_gbq.to_gbq(
    df,
    table_id,
    project_id=project_id,
    if_exists='replace'
    )

print("Data successfully loaded to BQ")

100%|██████████| 1/1 [00:00<00:00, 6105.25it/s]

Data successfully loaded to BQ
